In [1]:
using Turing
using LinearAlgebra
using Distributions
using Random

In [7]:
function prepare_var_data(Y::Matrix{Float64}, p::Int, X::Union{Matrix{Float64},Vector{Float64}} = Matrix{Float64}(undef, 0, 0), add_intercept::Bool = false)
    T, n = size(Y)
    Y_lagged = zeros(T - p, n * p)
    for t in (p + 1):T
        Y_lagged[t - p, :] = Y[t-p:t-1, :]'
    end

    predictors = Y_lagged

    if !isempty(X)
        if size(X, 1) != T
            error("The number of rows in X must be equal to the number of rows in Y.")
        end
        X_subset = X[p+1:end, :]
        predictors = hcat(predictors, X_subset)
    end

    if add_intercept
        intercept = ones(T - p, 1)
        predictors = hcat(intercept, predictors)
    end

    return Y[p+1:end, :], predictors
end

prepare_var_data (generic function with 3 methods)

In [28]:
using Test

@testset "prepare_var_data Tests" begin

    # Test case 1: Basic VAR data preparation (no exogenous variables, no intercept)
    Y = [1.0 2.0; 3.0 4.0; 5.0 6.0; 7.0 8.0]
    p1 = 1
    Y_obs1, predictors1 = prepare_var_data(Y, p1)
    @test size(Y_obs1) == (3, 2)
    @test size(predictors1) == (3, 2)
    @test predictors1[1, :] == [1.0, 2.0]
    @test predictors1[2, :] == [3.0, 4.0]
    @test predictors1[3, :] == [5.0, 6.0]
    @test Y_obs1[1, :] == [3.0, 4.0]
    @test Y_obs1[2, :] == [5.0, 6.0]
    @test Y_obs1[3, :] == [7.0, 8.0] 
   

    #Test case 2: VAR data preparation with 2 lags
    p2 = 2
    Y_obs2, predictors2 = prepare_var_data(Y, p2)
    @test size(Y_obs2) == (2, 2)
    @test size(predictors2) == (2, 4)
    @test predictors2[1, :] == [1.0, 2.0, 3.0, 4.0]
    @test predictors2[2, :] == [3.0, 4.0, 5.0, 6.0]   
    @test Y_obs2[1, :] == [5.0, 6.0]
    @test Y_obs2[2, :] == [7.0, 8.0]
   

   # Test case 3: VAR data preparation with exogenous variables
    X3 = [0.1, 0.2, 0.3, 0.4]
    p3 = 1
    Y_obs3, predictors3 = prepare_var_data(Y, p3, X3)
    @test size(Y_obs3) == (3, 2)
    @test size(predictors3) == (3, 3)
    @test predictors3[1, :] == [1.0, 2.0, 0.2]
    @test predictors3[2, :] == [3.0, 4.0, 0.3]
    @test predictors3[3, :] == [5.0, 6.0, 0.4]
    @test Y_obs1[1, :] == [3.0, 4.0]
    @test Y_obs1[2, :] == [5.0, 6.0]
    @test Y_obs1[3, :] == [7.0, 8.0]  
    
    
    # Test case 4: VAR data preparation with intercept
    p4 = 1
    Y_obs4, predictors4 = prepare_var_data(Y, p4, Matrix{Float64}(undef, 0, 0), true)
    @test size(Y_obs4) == (3, 2)
    @test size(predictors4) == (3, 3)
    @test predictors4[1, :] == [1.0, 1.0, 2.0]
    @test predictors4[2, :] == [1.0, 3.0, 4.0]
    @test predictors4[3, :] == [1.0, 5.0, 6.0]
    

    # Test case 5: VAR data preparation with exogenous variables and intercept
    X5 = [0.1 0.2; 0.3 0.4; 0.5 0.6; 0.7 0.8] 
    p5 = 1
    Y_obs5, predictors5 = prepare_var_data(Y, p5, X5, true)
    @test size(Y_obs5) == (3, 2)
    @test size(predictors5) == (3, 5)
    @test predictors5[1, :] == [1.0, 1.0, 2.0, 0.3, 0.4]
    @test predictors5[2, :] == [1.0, 3.0, 4.0, 0.5, 0.6]
    @test predictors5[3, :] == [1.0, 5.0, 6.0, 0.7, 0.8]
   
   

    # Test case 6: Error if X has incorrect number of rows
    X6 = [0.1, 0.2]
    p6 = 1
    @test_throws ErrorException prepare_var_data(Y, p6, X6) 
end

Test Summary:          | Pass  Total  Time
prepare_var_data Tests |   33     33  0.0s


Test.DefaultTestSet("prepare_var_data Tests", Any[], 33, false, false, true, 1.746994099196e9, 1.746994099196e9, false, "c:\\Users\\matsz\\programowanie\\Optymalizacja_portfela\\FinancialPlanner\\_wip\\VARs\\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl")

In [98]:
@model function var_model(Y::Matrix{Float64}, predictors::Matrix{Float64}, p::Int, num_exo::Int = 0)
    T_eff, num_regressors = size(predictors)
    n = size(Y, 2)

    # Priors for the coefficients (Normal)
    μ_β ~ MvNormal(zeros(num_regressors * n), 5.0 * I(num_regressors * n))    
    β = reshape(μ_β, n, num_regressors) # Reshape to n x (pn + k) matrix

    # Prior for the covariance matrix (Inverse Wishart)
    Σ ~ InverseWishart(T_eff-num_regressors, Matrix(I(n * n)))

    # Likelihood
    for t in 1:T_eff
        #yt = Y[t, :]
        xt = predictors[t, :]
        Y[t, :] ~ MvNormal(β * xt, Σ)
    end

    return β, Σ
end


var_model (generic function with 4 methods)

In [42]:
# Example Usage:
# Generate some sample data
T = 100  # Number of time points
n = 2    # Number of variables
p = 2    # Number of lags
k = 1    # Number of exogenous variables

# True parameters (for simulation)
true_β_var = [0.5 0.1 -0.2 0.8; 0.3 -0.1 0.2 0.4]
true_β_exo = [0.7; -0.5]
true_β = hcat(true_β_var, true_β_exo)
true_Ω = [1.0 0.2; 0.2 1.5]

X = randn(T, k)
Y = zeros(T, n)
Y[1:p, :] = randn(p, n) # Initialize with random values

for t in (p + 1):T
    Y_lagged = vec(Y[t-p:t-1, :])
    predictors = vcat(Y_lagged, X[t, :])
    Y[t, :] = true_β * predictors + rand(MvNormal(zeros(n), true_Ω))
end

In [99]:
# Sample
model = var_model(Y, X, p, k)
chain = sample(model, NUTS(), 1000)

DimensionMismatch: DimensionMismatch: The dimensions of mu and Sigma are inconsistent.

In [97]:
i = rand(InverseWishart(4, Matrix(I(2))), 1000)

var(i)


2×2 Matrix{Float64}:
 66.6255  10.3749
 10.3749   5.57817